In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score
from lightgbm import LGBMClassifier

In [26]:
text_features_path = 'text_features.csv'

text_features = pd.read_csv(text_features_path)
text_features.head()

,Unnamed: 0,bert_features,function (Function Words),pronoun (Pronouns),ppron (Personal Pronouns),i (I),we (We),you (You),shehe (SheHe),they (They),ipron (Impersonal Pronouns),article (Articles),prep (Prepositions),auxverb (Auxiliary Verbs),adverb (Adverbs),conj (Conjunctions),negate (Negations),verb (Verbs),adj (Adjectives),compare (Comparisons),interrog (Interrogatives),number (Numbers),quant (Quantifiers),affect (Affect),posemo (Positive Emotions),negemo (Negative Emotions),anx (Anx),anger (Anger),sad (Sad),social (Social),family (Family),friend (Friends),female (Female),male (Male),cogproc (Cognitive Processes),insight (Insight),cause (Causal),discrep (Discrepancies),tentat (Tentative),certain (Certainty),...,es_0_sim,es_0_rank,es_1_sim,es_1_rank,es_2_sim,es_2_rank,es_3_sim,es_3_rank,es_4_sim,es_4_rank,es_5_sim,es_5_rank,es_6_sim,es_6_rank,es_7_sim,es_7_rank,es_8_sim,es_8_rank,es_9_sim,es_9_rank,ru_0_sim,ru_0_rank,ru_1_sim,ru_1_rank,ru_2_sim,ru_2_rank,ru_3_sim,ru_3_rank,ru_4_sim,ru_4_rank,ru_5_sim,ru_5_rank,ru_6_sim,ru_6_rank,ru_7_sim,ru_7_rank,ru_8_sim,ru_8_rank,ru_9_sim,ru_9_rank
0,0,[-0.6696718 -0.6551779 -0.99718195 0.610286...,185,36,23,0,0,0,17,6,13,21,63,27,12,33,2,59,24,14,2,0,7,9,8,1,0,1,0,36,1,0,0,17,30,6,3,9,7,2,...,0.706539,6681,0.751676,1173.0,0.653933,1.294880e+05,0.675280,2475.0,0.732695,2.673000e+03,0.672330,1366.0,0.740247,31790.0,0.662952,4753.0,NaN,NaN,NaN,NaN,0.615500,1480671.0,0.603849,620.0,0.571263,67544.0,0.605179,365844.0,0.707031,10771.0,0.786291,100583.0,0.609681,80152.0,0.667345,21506.0,0.615025,1485.0,NaN,NaN
1,1,[-1.86803266e-02 -3.85523289e-01 -9.36701298e-...,74,19,12,0,0,0,12,0,7,10,18,16,7,8,3,29,4,2,1,0,1,1,1,0,0,0,0,19,0,1,12,1,17,3,2,1,8,1,...,0.695641,3523,0.665969,580.0,0.694328,4.546000e+04,0.734907,186331.0,0.695436,6.778000e+03,0.685045,12611.0,0.691760,46744.0,NaN,NaN,NaN,NaN,NaN,NaN,0.598045,62327.0,0.649774,11145.0,0.615232,11145.0,0.637109,42097.0,0.690500,48.0,0.622169,81315.0,0.731784,22.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,[-5.27673781e-01 -4.12977397e-01 -5.36640704e-...,144,29,22,1,1,1,19,0,7,22,46,15,18,18,1,49,8,6,3,5,7,9,7,2,0,2,0,40,7,1,15,7,44,9,7,10,5,7,...,0.736535,9,0.736535,9.0,0.642375,9.223372e+18,0.689069,15605.0,0.665570,1.396200e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.651085,263814.0,0.730419,2193880.0,0.705113,1974.0,0.665594,9187.0,0.620239,4003.0,0.691262,693315.0,0.632577,684775.0,0.660828,615260.0,0.689651,10669.0,0.656184,2491.0
3,3,[-2.57600158e-01 -2.91617334e-01 -3.44029903e-...,195,48,31,1,1,3,25,1,17,33,54,33,13,18,0,76,16,9,4,1,7,29,15,13,2,6,1,54,0,0,25,0,46,10,10,11,11,1,...,0.687738,13,0.699868,13.0,0.821108,1.173000e+03,0.733232,65.0,0.779260,9.223372e+18,0.643446,482.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.623543,13.0,0.619239,5263.0,0.641599,5263.0,0.642661,3043.0,0.724986,88.0,0.678961,5527.0,0.645147,482.0,0.703757,20483.0,NaN,NaN,NaN,NaN
4,4,[-0.5991494 -0.7292267 -0.97551596 0.669945...,157,14,6,0,0,0,4,2,8,37,47,27,6,22,7,47,16,11,2,3,5,20,10,10,0,8,0,28,9,0,13,0,36,4,7,1,7,6,...,0.577999,13,0.798768,139.0,0.755254,1.390000e+02,0.857150,3864.0,0.782987,4.977000e+04,0.781578,32739.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.577999,13.0,0.589259,1117.0,0.600724,11646.0,0.599963,365724.0,0.582942,11145.0,0.601203,620.0,0.671526,31887.0,0.576362,10083.0,NaN,NaN,NaN,NaN


In [27]:
text_features.shape

(500, 180)

In [28]:
text_features = text_features.drop(columns=['Unnamed: 0', 'index'])

In [46]:
tfwb = text_features.drop(columns='bert_features')
tfwb.head()

,function (Function Words),pronoun (Pronouns),ppron (Personal Pronouns),i (I),we (We),you (You),shehe (SheHe),they (They),ipron (Impersonal Pronouns),article (Articles),prep (Prepositions),auxverb (Auxiliary Verbs),adverb (Adverbs),conj (Conjunctions),negate (Negations),verb (Verbs),adj (Adjectives),compare (Comparisons),interrog (Interrogatives),number (Numbers),quant (Quantifiers),affect (Affect),posemo (Positive Emotions),negemo (Negative Emotions),anx (Anx),anger (Anger),sad (Sad),social (Social),family (Family),friend (Friends),female (Female),male (Male),cogproc (Cognitive Processes),insight (Insight),cause (Causal),discrep (Discrepancies),tentat (Tentative),certain (Certainty),differ (Differentiation),percept (Perceptual Processes),...,es_0_sim,es_0_rank,es_1_sim,es_1_rank,es_2_sim,es_2_rank,es_3_sim,es_3_rank,es_4_sim,es_4_rank,es_5_sim,es_5_rank,es_6_sim,es_6_rank,es_7_sim,es_7_rank,es_8_sim,es_8_rank,es_9_sim,es_9_rank,ru_0_sim,ru_0_rank,ru_1_sim,ru_1_rank,ru_2_sim,ru_2_rank,ru_3_sim,ru_3_rank,ru_4_sim,ru_4_rank,ru_5_sim,ru_5_rank,ru_6_sim,ru_6_rank,ru_7_sim,ru_7_rank,ru_8_sim,ru_8_rank,ru_9_sim,ru_9_rank
0,185,36,23,0,0,0,17,6,13,21,63,27,12,33,2,59,24,14,2,0,7,9,8,1,0,1,0,36,1,0,0,17,30,6,3,9,7,2,7,3,...,0.706539,6681,0.751676,1173.0,0.653933,1.294880e+05,0.675280,2475.0,0.732695,2.673000e+03,0.672330,1366.0,0.740247,31790.0,0.662952,4753.0,NaN,NaN,NaN,NaN,0.615500,1480671.0,0.603849,620.0,0.571263,67544.0,0.605179,365844.0,0.707031,10771.0,0.786291,100583.0,0.609681,80152.0,0.667345,21506.0,0.615025,1485.0,NaN,NaN
1,74,19,12,0,0,0,12,0,7,10,18,16,7,8,3,29,4,2,1,0,1,1,1,0,0,0,0,19,0,1,12,1,17,3,2,1,8,1,5,4,...,0.695641,3523,0.665969,580.0,0.694328,4.546000e+04,0.734907,186331.0,0.695436,6.778000e+03,0.685045,12611.0,0.691760,46744.0,NaN,NaN,NaN,NaN,NaN,NaN,0.598045,62327.0,0.649774,11145.0,0.615232,11145.0,0.637109,42097.0,0.690500,48.0,0.622169,81315.0,0.731784,22.0,NaN,NaN,NaN,NaN,NaN,NaN
2,144,29,22,1,1,1,19,0,7,22,46,15,18,18,1,49,8,6,3,5,7,9,7,2,0,2,0,40,7,1,15,7,44,9,7,10,5,7,9,4,...,0.736535,9,0.736535,9.0,0.642375,9.223372e+18,0.689069,15605.0,0.665570,1.396200e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.651085,263814.0,0.730419,2193880.0,0.705113,1974.0,0.665594,9187.0,0.620239,4003.0,0.691262,693315.0,0.632577,684775.0,0.660828,615260.0,0.689651,10669.0,0.656184,2491.0
3,195,48,31,1,1,3,25,1,17,33,54,33,13,18,0,76,16,9,4,1,7,29,15,13,2,6,1,54,0,0,25,0,46,10,10,11,11,1,5,17,...,0.687738,13,0.699868,13.0,0.821108,1.173000e+03,0.733232,65.0,0.779260,9.223372e+18,0.643446,482.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.623543,13.0,0.619239,5263.0,0.641599,5263.0,0.642661,3043.0,0.724986,88.0,0.678961,5527.0,0.645147,482.0,0.703757,20483.0,NaN,NaN,NaN,NaN
4,157,14,6,0,0,0,4,2,8,37,47,27,6,22,7,47,16,11,2,3,5,20,10,10,0,8,0,28,9,0,13,0,36,4,7,1,7,6,14,7,...,0.577999,13,0.798768,139.0,0.755254,1.390000e+02,0.857150,3864.0,0.782987,4.977000e+04,0.781578,32739.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.577999,13.0,0.589259,1117.0,0.600724,11646.0,0.599963,365724.0,0.582942,11145.0,0.601203,620.0,0.671526,31887.0,0.576362,10083.0,NaN,NaN,NaN,NaN


In [48]:
image_features_path = 'subsample.csv'
image_features = pd.read_csv(image_features_path)
image_features.head()

,filename,en,fr,de,es,ru,label,en-de,en-es,en-fr,en-ru
0,024fake.txt,Justin Bieber pokes fun at himself after stepp...,Justin Bieber se moque de lui-même après avoir...,"Justin Bieber macht sich lustig an sich, nachd...",Justin Bieber se burla de sí mismo después de ...,"Джастин Бибер высмеивает себя после того, как ...",1,0.000000,0.603129,0.593135,0.628361
1,002fake.txt,Brad Pitt Texts Jennifer Aniston Nonstop: Seek...,Brad Pitt envoie des SMS à Jennifer Aniston sa...,Brad Pitt Texte Jennifer Aniston Nonstop: such...,Brad Pitt texts Jennifer Aniston Nonstop: ¡bus...,Брэд Питт текстов Дженнифер Энистон Неустановк...,1,0.676341,0.598541,0.665803,0.679312
2,014fake.txt,Kanye West -- 'Nervous Breakdown',KANYE WEST - «Débarquement nerveux».,Kanye West - 'Nervenzusammenbruch'.,Kanye West - 'Guponado nervioso'.,Канье Уэст - «нервный срыв».,1,0.765920,0.741485,0.716170,0.755794
3,021fake.txt,Are Taylor Swift and Jake Gyllenhaal Dating Ag...,Taylor Swift et Jake Gyllenhaal se dressent à ...,Sind Taylor Swift und Jake Gyllenhaal wieder z...,¿Taylor Swift y Jake Gyllenhaal salen de nuevo?.,Тейлор Свифт и Джейк Джилленхал снова встречаю...,1,0.701117,0.691496,0.696562,0.659121
4,013fake.txt,Kanye West Wants to Enter Cosmetics Business L...,Kanye West veut entrer dans le secteur des cos...,Kanye West will wie Kylie Kosmetikgeschäft bet...,Kanye West quiere ingresar a un negocio de cos...,"Канье Уэст хочет войти в косметический бизнес,...",1,0.651074,0.742563,0.681806,0.703384


In [49]:
tfwb = tfwb[tfwb.file.isin(image_features.filename)]
tfwb.index = tfwb.file
target = tfwb.fake

In [62]:
image_features.index = image_features.filename

In [63]:
features = pd.concat([
    tfwb.drop(columns=['headline', 'file', 'content', 'fake']),
    image_features.drop(columns=['filename', 'en', 'ru', 'fr', 'de', 'es', 'label'])
    ], axis=1
)

In [64]:
features.head()

,function (Function Words),pronoun (Pronouns),ppron (Personal Pronouns),i (I),we (We),you (You),shehe (SheHe),they (They),ipron (Impersonal Pronouns),article (Articles),prep (Prepositions),auxverb (Auxiliary Verbs),adverb (Adverbs),conj (Conjunctions),negate (Negations),verb (Verbs),adj (Adjectives),compare (Comparisons),interrog (Interrogatives),number (Numbers),quant (Quantifiers),affect (Affect),posemo (Positive Emotions),negemo (Negative Emotions),anx (Anx),anger (Anger),sad (Sad),social (Social),family (Family),friend (Friends),female (Female),male (Male),cogproc (Cognitive Processes),insight (Insight),cause (Causal),discrep (Discrepancies),tentat (Tentative),certain (Certainty),differ (Differentiation),percept (Perceptual Processes),...,es_2_sim,es_2_rank,es_3_sim,es_3_rank,es_4_sim,es_4_rank,es_5_sim,es_5_rank,es_6_sim,es_6_rank,es_7_sim,es_7_rank,es_8_sim,es_8_rank,es_9_sim,es_9_rank,ru_0_sim,ru_0_rank,ru_1_sim,ru_1_rank,ru_2_sim,ru_2_rank,ru_3_sim,ru_3_rank,ru_4_sim,ru_4_rank,ru_5_sim,ru_5_rank,ru_6_sim,ru_6_rank,ru_7_sim,ru_7_rank,ru_8_sim,ru_8_rank,ru_9_sim,ru_9_rank,en-de,en-es,en-fr,en-ru
013fake.txt,44,11,4,0,1,0,3,0,7,6,18,6,1,3,0,15,4,0,2,1,2,7,6,1,0,1,0,12,3,1,2,3,13,0,5,1,3,1,3,4,...,0.799851,46792.0,0.808896,31790.0,0.756984,6778.0,0.719924,9850.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.791944,20483.0,0.667994,10083.0,0.657292,10083.0,0.667141,66472.0,0.671920,6617185.0,0.665086,222639.0,0.637954,13.0,0.667434,229202.0,0.812035,32961.0,NaN,NaN,0.651074,0.742563,0.681806,0.703384
019fake.txt,75,13,7,0,0,0,5,2,6,10,23,18,6,9,2,28,2,1,4,0,0,6,2,4,2,2,1,21,5,1,3,2,12,3,2,2,4,1,5,5,...,0.749515,114556.0,0.620593,924096.0,0.667464,482.0,0.667464,482.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.642325,1485.0,0.631223,620.0,0.644425,445.0,0.611833,13.0,0.614156,13.0,0.640806,1117.0,0.645223,1117.0,0.713397,88.0,NaN,NaN,NaN,NaN,0.737337,0.725466,0.740687,0.000000
015fake.txt,101,11,7,0,0,0,7,0,4,29,31,16,9,8,3,34,10,8,2,1,3,7,6,1,0,1,0,18,0,1,0,8,19,3,6,3,1,1,6,5,...,0.672255,88.0,0.619964,2664.0,0.716664,75813.0,0.639079,13.0,0.821596,232.0,0.742602,441.0,NaN,NaN,NaN,NaN,0.653466,2548.0,0.716102,88.0,0.656242,4907.0,0.658177,4907.0,0.646252,5663.0,0.652506,1117.0,0.662127,445.0,0.631096,13.0,NaN,NaN,NaN,NaN,0.442777,0.697727,0.687975,0.592353
020fake.txt,131,36,24,0,2,6,15,1,12,6,35,35,14,13,7,61,9,5,5,0,2,11,4,7,4,2,0,34,1,0,6,9,40,7,4,9,7,6,13,2,...,0.741728,580.0,0.721011,580.0,0.684920,4753.0,0.707332,13.0,0.768759,1366.0,0.792396,88.0,NaN,NaN,NaN,NaN,0.720697,88.0,0.740291,88.0,0.723531,2548.0,0.715681,2548.0,0.658573,1485.0,0.689100,9424.0,0.646324,620.0,0.630674,1395.0,NaN,NaN,NaN,NaN,0.456558,0.756588,0.777999,0.783917
012fake.txt,114,29,21,0,1,3,16,1,8,17,33,17,9,12,1,39,7,2,6,1,3,10,6,4,2,0,1,32,2,0,2,15,23,5,4,8,4,1,3,3,...,0.752607,186331.0,0.752350,13895.0,0.629275,2664.0,0.729571,39851.0,0.713095,13938.0,0.717831,1750.0,NaN,NaN,NaN,NaN,0.653068,1395.0,0.712399,88.0,0.736846,88.0,0.700171,9424.0,0.652980,620.0,0.665686,696.0,0.622032,837450.0,0.620446,3425152.0,0.621274,187283.0,0.619624,731733.0,0.601848,0.625723,0.000000,0.000000


In [60]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, shuffle=True, random_state=1777)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=1/3, random_state=1777)

In [61]:
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)
results = lgbm.predict(X_test)